Updated checkpoints: https://disk.yandex.ru/d/cK1PQ1-nn3PbKg

# Домашнее задание

1) В этой домашней работе вам предстоит решить задачу сегментации объект/фон на датасете PascalVoc2012. . Это такая же задача, как мы решали на семинаре, поэтому пользуйтесь наработками и рекомендациями
2) необходимо дополнить список аугментаций (>= 2) : https://pytorch.org/vision/master/transforms.html , https://albumentations.ai
3) Добавить понятные метрики на валидацию mean-IoU 
4) Добавить лосс Jaccard loss или DICE loss к BCE: https://arxiv.org/pdf/2006.14822.pdf
5) Взять непредобученную модель(можно использовать UNET из семинара, взять другую, реализовать свою) 
6) Пообучать модель несколько эпох 
7) Показать результаты работы - графики mean-IoU, лосс и примеры работы


In [14]:
data_path = "data"
checkpoints_path = f"{data_path}/models"
checkpoints_file = "checkpoints.zip"

# Prepare data

In [ ]:
"""!wget https://data.deepai.org/PascalVOC2012.zip
!unzip PascalVOC2012.zip
!mv voc2012 VOCdevkit
!pip install segmentation_models_pytorch
!python -m pip install pytorch-lightning
!git clone -b autumn_2023 https://github.com/AlekseySpasenov/dl-course.git"""

In [15]:
import os
from zipfile import ZipFile
if not os.path.exists(checkpoints_path):
    raise FileNotFoundError("Скачайте архив с обученными моделями")

with ZipFile(f"{checkpoints_path}/{checkpoints_file}", "r") as zf:
    zf.extractall(checkpoints_path)